In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

#for save & load pickle
import sys
import os
import pickle

def save_as_pickled_object(obj, filepath):
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj, protocol=pickle.HIGHEST_PROTOCOL)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])


def try_to_load_as_pickled_object_or_None(filepath):
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


import os
PATH = "../csv/"
print(os.listdir(PATH))

['application_test.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
#read csv
bureau_balance = import_data(PATH+'bureau_balance.csv')
bureau = import_data(PATH+'bureau.csv')

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 156.21 MB
Decreased by 75.0%
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 78.57 MB
Decreased by 64.7%


In [4]:
#bureau balance
df1=bureau_balance.sort_values("SK_ID_BUREAU")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_BUREAU","STATUS"]).agg(['sum','max','mean'])
df2.head(10)

MONTHS_BALANCE                COUNT         
                               sum max       mean   sum max mean
SK_ID_BUREAU STATUS                                             
5001709      C             -3655.0   0 -42.500000    86   1    1
             X             -1001.0 -86 -91.000000    11   1    1
5001710      0              -254.0 -48 -50.800000     5   1    1
             C             -1128.0   0 -23.500000    48   1    1
             X             -2021.0 -49 -67.366667    30   1    1
5001711      0                -6.0  -1  -2.000000     3   1    1
             X                 0.0   0   0.000000     1   1    1
5001712      0              -135.0  -9 -13.500000    10   1    1
             C               -36.0   0  -4.000000     9   1    1
5001713      X              -231.0   0 -10.500000    22   1    1

In [5]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("bb_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['bb_MONTHS_BALANCE_sum', 'bb_MONTHS_BALANCE_max', 'bb_MONTHS_BALANCE_mean', 'bb_COUNT_sum', 'bb_COUNT_max', 'bb_COUNT_mean']


bb_MONTHS_BALANCE_sum  bb_MONTHS_BALANCE_max  \
SK_ID_BUREAU STATUS                                                 
5001709      C                     -3655.0                      0   
             X                     -1001.0                    -86   
5001710      0                      -254.0                    -48   
             C                     -1128.0                      0   
             X                     -2021.0                    -49   

                     bb_MONTHS_BALANCE_mean  bb_COUNT_sum  bb_COUNT_max  \
SK_ID_BUREAU STATUS                                                       
5001709      C                   -42.500000            86             1   
             X                   -91.000000            11             1   
5001710      0                   -50.800000             5             1   
             C                   -23.500000            48             1   
             X                   -67.366667            30             1   

                     bb_COUNT_mean  
SK_ID_BUREAU STATUS                 
5001709      C                   1  
             X                   1  
5001710      0                   1  
             C                   1  
             X                   1

In [6]:
df2_0 = df2.xs(key="0",level=1)
df2_0.columns = df2_0.columns + "_0"

df2_1 = df2.xs(key="1",level=1)
df2_1.columns = df2_1.columns + "_1"

df2_2 = df2.xs(key="2",level=1)
df2_2.columns = df2_2.columns + "_2"

df2_3 = df2.xs(key="3",level=1)
df2_3.columns = df2_3.columns + "_3"

df2_4 = df2.xs(key="4",level=1)
df2_4.columns = df2_4.columns + "_4"

df2_5 = df2.xs(key="5",level=1)
df2_5.columns = df2_5.columns + "_5"

df2_C = df2.xs(key="C",level=1)
df2_C.columns = df2_C.columns + "_C"

df2_X = df2.xs(key="X",level=1)
df2_X.columns = df2_X.columns + "_X"

In [7]:
df3= pd.merge(df2_0, df2_1, left_index=True, right_index=True)
df3= pd.merge(df3, df2_2, left_index=True, right_index=True)
df3= pd.merge(df3, df2_3, left_index=True, right_index=True)
df3= pd.merge(df3, df2_4, left_index=True, right_index=True)
df3= pd.merge(df3, df2_5, left_index=True, right_index=True)
df3= pd.merge(df3, df2_C, left_index=True, right_index=True)
df3= pd.merge(df3, df2_X, left_index=True, right_index=True)

In [8]:
pd.set_option('display.max_columns', 300)
df_bureau_balance = df3.reset_index()
df_bureau_balance.head()

,SK_ID_BUREAU,bb_MONTHS_BALANCE_sum_0,bb_MONTHS_BALANCE_max_0,bb_MONTHS_BALANCE_mean_0,bb_COUNT_sum_0,bb_COUNT_max_0,bb_COUNT_mean_0,bb_MONTHS_BALANCE_sum_1,bb_MONTHS_BALANCE_max_1,bb_MONTHS_BALANCE_mean_1,bb_COUNT_sum_1,bb_COUNT_max_1,bb_COUNT_mean_1,bb_MONTHS_BALANCE_sum_2,bb_MONTHS_BALANCE_max_2,bb_MONTHS_BALANCE_mean_2,bb_COUNT_sum_2,bb_COUNT_max_2,bb_COUNT_mean_2,bb_MONTHS_BALANCE_sum_3,bb_MONTHS_BALANCE_max_3,bb_MONTHS_BALANCE_mean_3,bb_COUNT_sum_3,bb_COUNT_max_3,bb_COUNT_mean_3,bb_MONTHS_BALANCE_sum_4,bb_MONTHS_BALANCE_max_4,bb_MONTHS_BALANCE_mean_4,bb_COUNT_sum_4,bb_COUNT_max_4,bb_COUNT_mean_4,bb_MONTHS_BALANCE_sum_5,bb_MONTHS_BALANCE_max_5,bb_MONTHS_BALANCE_mean_5,bb_COUNT_sum_5,bb_COUNT_max_5,bb_COUNT_mean_5,bb_MONTHS_BALANCE_sum_C,bb_MONTHS_BALANCE_max_C,bb_MONTHS_BALANCE_mean_C,bb_COUNT_sum_C,bb_COUNT_max_C,bb_COUNT_mean_C,bb_MONTHS_BALANCE_sum_X,bb_MONTHS_BALANCE_max_X,bb_MONTHS_BALANCE_mean_X,bb_COUNT_sum_X,bb_COUNT_max_X,bb_COUNT_mean_X
0,5004027,-390.0,-72,-78.000000,5,1,1,-77.0,-77,-77.000000,1,1,1,-147.0,-71,-73.500000,2,1,1,-145.0,-70,-72.5,2,1,1,-143.0,-69,-71.5,2,1,1,-403.0,-64,-67.166667,6,1,1,-2016.0,0,-31.5,64,1,1,-82.0,-82,-82.000000,1,1,1
1,5017063,-287.0,-26,-31.888889,9,1,1,-199.0,-21,-28.428571,7,1,1,-65.0,-20,-21.666667,3,1,1,-19.0,-19,-19.0,1,1,1,-18.0,-18,-18.0,1,1,1,-17.0,-17,-17.000000,1,1,1,-136.0,0,-8.0,17,1,1,-3915.0,-39,-67.500000,58,1,1
2,5023458,-1663.0,-7,-57.344828,29,1,1,-78.0,-14,-39.000000,2,1,1,-76.0,-13,-38.000000,2,1,1,-62.0,-62,-62.0,1,1,1,-61.0,-61,-61.0,1,1,1,-1694.0,-17,-38.500000,44,1,1,-18.0,-3,-4.5,4,1,1,-86.0,-86,-86.000000,1,1,1
3,5024940,-89.0,-89,-89.000000,1,1,1,-175.0,-87,-87.500000,2,1,1,-86.0,-86,-86.000000,1,1,1,-85.0,-85,-85.0,1,1,1,-167.0,-83,-83.5,2,1,1,-82.0,-82,-82.000000,1,1,1,-1653.0,0,-28.5,58,1,1,-1668.0,-58,-69.500000,24,1,1
4,5025070,-2445.0,-15,-50.937500,48,1,1,-177.0,-44,-59.000000,3,1,1,-68.0,-68,-68.000000,1,1,1,-67.0,-67,-67.0,1,1,1,-66.0,-66,-66.0,1,1,1,-65.0,-65,-65.000000,1,1,1,-91.0,0,-6.5,14,1,1,-424.0,-14,-30.285714,14,1,1


In [9]:
#bureau
df1 = pd.merge(bureau, df_bureau_balance, on="SK_ID_BUREAU",how='left')
df1 = df1.drop(columns="SK_ID_BUREAU")
df1 = df1.sort_values("SK_ID_CURR")
df1["COUNT"] = 1

In [10]:
df2 = df1.groupby(["SK_ID_CURR","CREDIT_ACTIVE"]).agg(['sum','max','mean'])

In [11]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("bureau_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['bureau_DAYS_CREDIT_sum', 'bureau_DAYS_CREDIT_max', 'bureau_DAYS_CREDIT_mean', 'bureau_CREDIT_DAY_OVERDUE_sum', 'bureau_CREDIT_DAY_OVERDUE_max', 'bureau_CREDIT_DAY_OVERDUE_mean', 'bureau_DAYS_CREDIT_ENDDATE_sum', 'bureau_DAYS_CREDIT_ENDDATE_max', 'bureau_DAYS_CREDIT_ENDDATE_mean', 'bureau_DAYS_ENDDATE_FACT_sum', 'bureau_DAYS_ENDDATE_FACT_max', 'bureau_DAYS_ENDDATE_FACT_mean', 'bureau_AMT_CREDIT_MAX_OVERDUE_sum', 'bureau_AMT_CREDIT_MAX_OVERDUE_max', 'bureau_AMT_CREDIT_MAX_OVERDUE_mean', 'bureau_CNT_CREDIT_PROLONG_sum', 'bureau_CNT_CREDIT_PROLONG_max', 'bureau_CNT_CREDIT_PROLONG_mean', 'bureau_AMT_CREDIT_SUM_sum', 'bureau_AMT_CREDIT_SUM_max', 'bureau_AMT_CREDIT_SUM_mean', 'bureau_AMT_CREDIT_SUM_DEBT_sum', 'bureau_AMT_CREDIT_SUM_DEBT_max', 'bureau_AMT_CREDIT_SUM_DEBT_mean', 'bureau_AMT_CREDIT_SUM_LIMIT_sum', 'bureau_AMT_CREDIT_SUM_LIMIT_max', 'bureau_AMT_CREDIT_SUM_LIMIT_mean', 'bureau_AMT_CREDIT_SUM_OVERDUE_sum', 'bureau_AMT_CREDIT_SUM_OVERDUE_max', 'bureau_AMT_CREDIT_SUM_OVERDUE_mean',

bureau_DAYS_CREDIT_sum  bureau_DAYS_CREDIT_max  \
SK_ID_CURR CREDIT_ACTIVE                                                   
100001     Active                         -928.0                     -49   
           Closed                        -4217.0                    -857   
100002     Active                        -1145.0                    -103   
           Closed                        -5847.0                    -476   
100003     Active                         -606.0                    -606   

                          bureau_DAYS_CREDIT_mean  \
SK_ID_CURR CREDIT_ACTIVE                            
100001     Active                     -309.333333   
           Closed                    -1054.250000   
100002     Active                     -572.500000   
           Closed                     -974.500000   
100003     Active                     -606.000000   

                          bureau_CREDIT_DAY_OVERDUE_sum  \
SK_ID_CURR CREDIT_ACTIVE                                  
100001     Active                                     0   
           Closed                                     0   
100002     Active                                     0   
           Closed                                     0   
100003     Active                                     0   

                          bureau_CREDIT_DAY_OVERDUE_max  \
SK_ID_CURR CREDIT_ACTIVE                                  
100001     Active                                     0   
           Closed                                     0   
100002     Active                                     0   
           Closed                                     0   
100003     Active                                     0   

                          bureau_CREDIT_DAY_OVERDUE_mean  \
SK_ID_CURR CREDIT_ACTIVE                                   
100001     Active                                    0.0   
           Closed                                    0.0   
100002     Active                                    0.0   
           Closed                                    0.0   
100003     Active                                    0.0   

                          bureau_DAYS_CREDIT_ENDDATE_sum  \
SK_ID_CURR CREDIT_ACTIVE                                   
100001     Active                                 3092.0   
           Closed                                -2514.0   
100002     Active                                  780.0   
           Closed                                -2874.0   
100003     Active                                 1216.0   

                          bureau_DAYS_CREDIT_ENDDATE_max  \
SK_ID_CURR CREDIT_ACTIVE                                   
100001     Active                                 1778.0   
           Closed                                 -179.0   
100002     Active                                  780.0   
           Closed                                   85.0   
100003     Active                                 1216.0   

                          bureau_DAYS_CREDIT_ENDDATE_mean  \
SK_ID_CURR CREDIT_ACTIVE                                    
100001     Active                                  1030.0   
           Closed                                  -628.5   
100002     Active                                   780.0   
           Closed                                  -575.0   
100003     Active                                  1216.0   

                          bureau_DAYS_ENDDATE_FACT_sum  \
SK_ID_CURR CREDIT_ACTIVE                                 
100001     Active                                  0.0   
           Closed                              -3302.0   
100002     Active                                  0.0   
           Closed                              -4184.0   
100003     Active                                  0.0   

                          bureau_DAYS_ENDDATE_FACT_max  \
SK_ID_CURR CREDIT_ACTIVE                                 
100001     Active                                  NaN   
           Closed    

In [12]:
df1['CREDIT_ACTIVE'].value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64

In [13]:
df2_Active = df2.xs(key="Active",level=1)
df2_Active.columns = df2_Active.columns + "_Ac"

df2_Closed = df2.xs(key="Closed",level=1)
df2_Closed.columns = df2_Closed.columns + "_Cl"

df2_Sold= df2.xs(key="Sold",level=1)
df2_Sold.columns = df2_Sold.columns + "_So"

df2_Bad = df2.xs(key="Bad debt",level=1)
df2_Bad.columns = df2_Bad.columns + "_Ba"

In [14]:
df3= pd.merge(df2_Active, df2_Closed, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Sold, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Bad, left_index=True, right_index=True, how='outer')

In [15]:
bureau_data = df3.reset_index()
bureau_data.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_sum_Ac,bureau_DAYS_CREDIT_max_Ac,bureau_DAYS_CREDIT_mean_Ac,bureau_CREDIT_DAY_OVERDUE_sum_Ac,bureau_CREDIT_DAY_OVERDUE_max_Ac,bureau_CREDIT_DAY_OVERDUE_mean_Ac,bureau_DAYS_CREDIT_ENDDATE_sum_Ac,bureau_DAYS_CREDIT_ENDDATE_max_Ac,bureau_DAYS_CREDIT_ENDDATE_mean_Ac,bureau_DAYS_ENDDATE_FACT_sum_Ac,bureau_DAYS_ENDDATE_FACT_max_Ac,bureau_DAYS_ENDDATE_FACT_mean_Ac,bureau_AMT_CREDIT_MAX_OVERDUE_sum_Ac,bureau_AMT_CREDIT_MAX_OVERDUE_max_Ac,bureau_AMT_CREDIT_MAX_OVERDUE_mean_Ac,bureau_CNT_CREDIT_PROLONG_sum_Ac,bureau_CNT_CREDIT_PROLONG_max_Ac,bureau_CNT_CREDIT_PROLONG_mean_Ac,bureau_AMT_CREDIT_SUM_sum_Ac,bureau_AMT_CREDIT_SUM_max_Ac,bureau_AMT_CREDIT_SUM_mean_Ac,bureau_AMT_CREDIT_SUM_DEBT_sum_Ac,bureau_AMT_CREDIT_SUM_DEBT_max_Ac,bureau_AMT_CREDIT_SUM_DEBT_mean_Ac,bureau_AMT_CREDIT_SUM_LIMIT_sum_Ac,bureau_AMT_CREDIT_SUM_LIMIT_max_Ac,bureau_AMT_CREDIT_SUM_LIMIT_mean_Ac,bureau_AMT_CREDIT_SUM_OVERDUE_sum_Ac,bureau_AMT_CREDIT_SUM_OVERDUE_max_Ac,bureau_AMT_CREDIT_SUM_OVERDUE_mean_Ac,bureau_DAYS_CREDIT_UPDATE_sum_Ac,bureau_DAYS_CREDIT_UPDATE_max_Ac,bureau_DAYS_CREDIT_UPDATE_mean_Ac,bureau_AMT_ANNUITY_sum_Ac,bureau_AMT_ANNUITY_max_Ac,bureau_AMT_ANNUITY_mean_Ac,bureau_bb_MONTHS_BALANCE_sum_0_sum_Ac,bureau_bb_MONTHS_BALANCE_sum_0_max_Ac,bureau_bb_MONTHS_BALANCE_sum_0_mean_Ac,bureau_bb_MONTHS_BALANCE_max_0_sum_Ac,bureau_bb_MONTHS_BALANCE_max_0_max_Ac,bureau_bb_MONTHS_BALANCE_max_0_mean_Ac,bureau_bb_MONTHS_BALANCE_mean_0_sum_Ac,bureau_bb_MONTHS_BALANCE_mean_0_max_Ac,bureau_bb_MONTHS_BALANCE_mean_0_mean_Ac,bureau_bb_COUNT_sum_0_sum_Ac,bureau_bb_COUNT_sum_0_max_Ac,bureau_bb_COUNT_sum_0_mean_Ac,bureau_bb_COUNT_max_0_sum_Ac,bureau_bb_COUNT_max_0_max_Ac,bureau_bb_COUNT_max_0_mean_Ac,bureau_bb_COUNT_mean_0_sum_Ac,bureau_bb_COUNT_mean_0_max_Ac,bureau_bb_COUNT_mean_0_mean_Ac,bureau_bb_MONTHS_BALANCE_sum_1_sum_Ac,bureau_bb_MONTHS_BALANCE_sum_1_max_Ac,bureau_bb_MONTHS_BALANCE_sum_1_mean_Ac,bureau_bb_MONTHS_BALANCE_max_1_sum_Ac,bureau_bb_MONTHS_BALANCE_max_1_max_Ac,bureau_bb_MONTHS_BALANCE_max_1_mean_Ac,bureau_bb_MONTHS_BALANCE_mean_1_sum_Ac,bureau_bb_MONTHS_BALANCE_mean_1_max_Ac,bureau_bb_MONTHS_BALANCE_mean_1_mean_Ac,bureau_bb_COUNT_sum_1_sum_Ac,bureau_bb_COUNT_sum_1_max_Ac,bureau_bb_COUNT_sum_1_mean_Ac,bureau_bb_COUNT_max_1_sum_Ac,bureau_bb_COUNT_max_1_max_Ac,bureau_bb_COUNT_max_1_mean_Ac,bureau_bb_COUNT_mean_1_sum_Ac,bureau_bb_COUNT_mean_1_max_Ac,bureau_bb_COUNT_mean_1_mean_Ac,bureau_bb_MONTHS_BALANCE_sum_2_sum_Ac,bureau_bb_MONTHS_BALANCE_sum_2_max_Ac,bureau_bb_MONTHS_BALANCE_sum_2_mean_Ac,bureau_bb_MONTHS_BALANCE_max_2_sum_Ac,bureau_bb_MONTHS_BALANCE_max_2_max_Ac,bureau_bb_MONTHS_BALANCE_max_2_mean_Ac,bureau_bb_MONTHS_BALANCE_mean_2_sum_Ac,bureau_bb_MONTHS_BALANCE_mean_2_max_Ac,bureau_bb_MONTHS_BALANCE_mean_2_mean_Ac,bureau_bb_COUNT_sum_2_sum_Ac,bureau_bb_COUNT_sum_2_max_Ac,bureau_bb_COUNT_sum_2_mean_Ac,bureau_bb_COUNT_max_2_sum_Ac,bureau_bb_COUNT_max_2_max_Ac,bureau_bb_COUNT_max_2_mean_Ac,bureau_bb_COUNT_mean_2_sum_Ac,bureau_bb_COUNT_mean_2_max_Ac,bureau_bb_COUNT_mean_2_mean_Ac,bureau_bb_MONTHS_BALANCE_sum_3_sum_Ac,bureau_bb_MONTHS_BALANCE_sum_3_max_Ac,bureau_bb_MONTHS_BALANCE_sum_3_mean_Ac,bureau_bb_MONTHS_BALANCE_max_3_sum_Ac,bureau_bb_MONTHS_BALANCE_max_3_max_Ac,bureau_bb_MONTHS_BALANCE_max_3_mean_Ac,bureau_bb_MONTHS_BALANCE_mean_3_sum_Ac,bureau_bb_MONTHS_BALANCE_mean_3_max_Ac,bureau_bb_MONTHS_BALANCE_mean_3_mean_Ac,bureau_bb_COUNT_sum_3_sum_Ac,bureau_bb_COUNT_sum_3_max_Ac,bureau_bb_COUNT_sum_3_mean_Ac,bureau_bb_COUNT_max_3_sum_Ac,bureau_bb_COUNT_max_3_max_Ac,bureau_bb_COUNT_max_3_mean_Ac,bureau_bb_COUNT_mean_3_sum_Ac,bureau_bb_COUNT_mean_3_max_Ac,bureau_bb_COUNT_mean_3_mean_Ac,bureau_bb_MONTHS_BALANCE_sum_4_sum_Ac,bureau_bb_MONTHS_BALANCE_sum_4_max_Ac,bureau_bb_MONTHS_BALANCE_sum_4_mean_Ac,bureau_bb_MONTHS_BALANCE_max_4_sum_Ac,bureau_bb_MONTHS_BALANCE_max_4_max_Ac,bureau_bb_MONTHS_BALANCE_max_4_mean_Ac,bureau_bb_MONTHS_BALANCE_mean_4_sum_Ac,bureau_bb_MONTHS_BALANCE_mean_4_max_Ac,bureau_bb_MONTHS_BALANCE_mean_4_mean_Ac,b

In [16]:
#save pkl
save_as_pickled_object(bureau_data, '/Volumes/sub/kaggle/pkl/Home Credit Default Risk/bureau_data.pkl')